In [93]:
import sklearn
import pandas as pd
import numpy as np

In [94]:
data=pd.read_csv('/content/data_for_large_scale.csv')
data

,Feature-1,Feature-2,Feature-3,Feature-4,Feature-5,Feature-6,Feature-7,Feature-8,Feature-9,Feature-10,Target
0,-1.580,1.0500,1.0600,-0.440,0.45100,-0.0348,0.643,0.2650,0.268,-0.8510,84.70
1,-0.832,-0.8660,-1.3400,0.138,1.18000,0.7330,-1.410,0.1350,-0.088,-1.5500,-211.00
2,-0.237,2.0900,-3.9300,0.296,0.35200,-0.5010,0.961,-0.0287,1.820,0.9380,-96.90
3,-1.170,-1.1300,-1.0900,1.120,0.31200,0.1830,0.448,-0.8190,-1.010,-1.0800,-152.00
4,0.260,-0.0273,0.9250,-1.150,-1.39000,0.0251,0.627,0.0950,-0.280,-0.8480,-57.70
...,...,...,...,...,...,...,...,...,...,...,...
89995,-0.268,-0.3200,-0.0146,-0.251,1.76000,0.8590,-1.090,-1.3800,1.760,-1.0900,-4.15
89996,-0.774,0.4960,1.9400,0.673,0.29500,-0.0137,-0.966,-0.6270,-0.113,0.0767,63.50
89997,0.509,-0.3220,0.6810,-0.276,-0.00553,-1.1000,0.453,0.7270,-1.940,-2.0100,-80.20
89998,0.360,1.2500,0.6510,-0.871,0.37900,0.7690,1.330,-0.3670,-1.280,-1.1000,158.00


In [95]:
x=data.drop('Target',axis=1)
y=data['Target']

In [96]:
x.shape,y.shape

((90000, 10), (90000,))

In [97]:
x_array=x.to_numpy()
y_array=y.to_numpy()

In [98]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_array,y_array,test_size=0.3,random_state=10)

In [99]:
x_train.shape

(63000, 10)

In [100]:
x_train_reshaped=x_train.reshape(-1,90,10)
x_test_reshaped=x_test.reshape(-1,90,10)

In [101]:
y_train_reshaped=y_train.reshape(-1,90)
y_test_reshaped=y_test.reshape(-1,90)

In [102]:
x_test_reshaped.shape

(300, 90, 10)

In [103]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor(random_state=10)

In [104]:
for i in range(x_train_reshaped.shape[0]):
  x_batch=x_train_reshaped[i]
  y_batch=y_train_reshaped[i]
  sgd.partial_fit(x_batch,y_batch)

In [105]:
sgd.coef_

array([51.3212522 , 22.26535893, 81.23762868, 53.19615839, 76.46565732,
       71.47204651, 93.45092409, 51.92198178, 30.03910055, 40.95656774])

In [106]:
sgd.intercept_

array([-0.0054532])

In [107]:
from sklearn.metrics import r2_score
y_pred_list=[]

In [108]:
for i in range(x_test_reshaped.shape[0]):
  x_batch_test=x_test_reshaped[i]
  y_pred=sgd.predict(x_batch_test)
  y_pred_list.append(y_pred)


In [109]:
len(y_pred_list)

300

In [110]:
r2_score(y_test_reshaped,y_pred_list)

0.9999919041690718

In [111]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor(random_state=10)

In [112]:
for i in range(5):
  x_batch=x_train_reshaped[i]
  y_batch=y_train_reshaped[i]
  sgd.partial_fit(x_batch,y_batch)
  print(sgd.coef_)

[20.60004667  7.85628793 23.24335397 21.31751751 29.993019   24.20767021
 29.53600948 27.68379716 18.61115024  4.44061696]
[27.51045193 11.51321245 38.22831885 24.26093919 41.61489153 38.03337726
 47.41716622 30.85612682 18.76455626 11.01688182]
[31.71529495 15.74159069 44.17862746 34.04778846 48.57710653 45.71718856
 54.40287649 35.08594975 21.90181482 21.15602691]
[34.38246633 15.40994181 52.12945406 37.6878988  52.09758505 51.61424162
 62.83912511 41.4345424  23.75565095 25.52052546]
[38.10307994 16.55515794 55.39574282 39.59670638 56.0556394  54.99254453
 67.70253651 43.94562783 24.80852625 29.56006746]


In [113]:
data_url='https://archive.ics.uci.edu/ml/machine-learning-databases/00246/3D_spatial_network.txt'

In [114]:
data=pd.read_csv(data_url,chunksize=20000,iterator=True)

In [131]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor(random_state=10)
total_length=0

for i in pd.read_csv(data_url,chunksize=20000,iterator=True):
  x=i.iloc[:,1:3]##column no 1,2,3 is features
  y=i.iloc[:,3]##cplumn no 4 is label
  total_length+=x.shape[0]
  scaled_x=ss.fit_transform(x)##partial fit
  sgd.partial_fit(scaled_x,y)


In [124]:
total_length

434873

In [139]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor(random_state=10)
iter=1

for i in pd.read_csv(data_url,chunksize=20000,iterator=True):
  x=i.iloc[:,1:3]
  y=i.iloc[:,3]
  total_length+=x.shape[0]
  scaled_x=ss.fit_transform(x)
  sgd.partial_fit(scaled_x,y)
  if iter==7:
    print(sgd.intercept_)
    print(sgd.coef_)
    break
  iter+=1


[20.15784555]
[ 4.47749538 -5.00443859]


In [142]:
from sklearn.datasets import load_iris
x,y=load_iris(return_X_y=True,as_frame=True)

In [143]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [144]:
from sklearn.preprocessing import Normalizer
nr=Normalizer()

In [146]:
x_train_normalized=nr.fit_transform(x_train)
x_test_normalized=nr.fit_transform(x_test)

In [147]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score

In [148]:
knc_2=KNeighborsClassifier(n_neighbors=2)
knc_3=KNeighborsClassifier(n_neighbors=3)
knc_4=KNeighborsClassifier(n_neighbors=4)

In [153]:
knc_2.fit(x_train_normalized,y_train)
y_pred_2=knc_2.predict(x_test_normalized)
print(accuracy_score(y_test,y_pred_2))

0.9666666666666667


In [154]:
knc_3.fit(x_train_normalized,y_train)
y_pred_3=knc_2.predict(x_test_normalized)
print(accuracy_score(y_test,y_pred_3))

0.9666666666666667


In [155]:
knc_4.fit(x_train_normalized,y_train)
y_pred_4=knc_2.predict(x_test_normalized)
print(accuracy_score(y_test,y_pred_4))

0.9666666666666667


In [157]:
f1_score(y_test,y_pred_3,average='weighted')

0.9671111111111111